(cont:fs:layout)=
# File System Layout

To store a file system on a real disk, the high-level objects (directories, files, symbolic links) must be translated into read and write on blocks blocks identified by logical block addresses, how do we do that?  How do we even know what file system should be used to understand the blocks on the disk? How do we load the operating system in the first place off of the disk?

```{figure} ../images/fs/disklayout-partitions.png
---
width: 70%
name: fs:disklayout
---
Base disk layout with partition table
```

As shown in {numref}`fs:disklayout` the operating system normally writes a partition table at the beginning of the disk that can be used the divide the disk into *partitions* each potentially containing a different file system. For each partition, the table indicates the range of blocks in it, indicate if the partition contains an operating system that could be executed, and the type of file system that should be used to interpret the blocks. 

When a system starts up, the firmware will read the partition table, look at which partitions contain operating systems, and (potentially asking user) decide which one to boot.  It then reads the boot block of that partition, and start executing code from that block that will in turn read other blocks...


```{figure} ../images/fs/disklayout-withexfs.png
---
width: 70%
name: fs:disklayout-withFS
---
Example of a file system that might be in a partition. 
```

{numref}`fs:disklayout-withFS` shows example contents of a partition.  After the boot block, all the information is specific to the file system.  Once the operating system kernel is started, it will use the type of file system information from the partition table to figure out if it understands that kind of file system.
Recall, there are hundreds and hundreds of file systems, and operating systems like Linux have file system code specific to each one.  When we talked about [operating system structure](cont:gs:structure:struc) we talked about how the file system code is normally dynamically loaded (see {numref}`os-struc-services`); if the operating system had all the file systems it could possibly understand loaded, it would be huge. 

While every file system does it differently, {numref}`fs:disklayout-withFS` shows on example organization for a simple file system. Key elements on disk are:
- **Superblock**: that describes where the inodes are, where free block information is kept, the block size, the, the inode number for the root directory, ... and all kinds of information for how to recover the file system if failures occur.
- **Free info**: some data structure to describe which blocks in the partition/disk are free.
- **Inodes**: an array of inode information indexed by inode number
where the remainder of the partition is just disk blocks. 

```{sidebar} 
Note that instead of 512-byte sectors, file systems traditionally use
*disk blocks*, which are some small power-of-two multiple of the sector
size, typically 1KB, 2KB, or 4KB. Reading and writing is performed in
units of complete blocks, and addresses are stored as disk block numbers
rather than LBAs, and are then multiplied by the appropriate value
before being passed to the disk. Since modern disk drives have an
internal sector size of 4 KB (despite pretending to support 512-byte
sectors) and the virtual memory page size is 4 KB on most systems today,
that has become a very common file system block size.
```
the next four chapters describe key file system design decisions, namely how to:
1. [keep track of the blocks used in a file](fs:dl_track_used)
2. [keep track of the free blocks](fs:dl_track_free)                                                
3. [maintain the name space](fs:dl_name)
4. [deal with failures](fs:dl_failures)  